In [1]:
import pandas as pd
import numpy as np
import evaluate
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoTokenizer, TextClassificationPipeline
from datasets import Dataset

In [2]:
dfo = pd.read_csv('data/obama_cleaned.csv')
dfo = dfo.rename(columns={'tweets' : 'text', 'class' : 'label'})

dfr = pd.read_csv('data/romney_cleaned.csv')
dfr = dfr.rename(columns={'tweets' : 'text', 'class' : 'label'})

# Pre-trained model: BERTweet

Fine-tuning using our data:

In [3]:
Xo = dfo['text']
yo = dfo['label'].map({1 : 2, 0 : 1, -1 : 0})
Xo_train, Xo_eval, yo_train, yo_eval = train_test_split(Xo, yo, test_size = 0.25, random_state = 21)

Xr = dfr['text']
yr = dfr['label'].map({1 : 2, 0 : 1, -1 : 0})
Xr_train, Xr_eval, yr_train, yr_eval = train_test_split(Xr, yr, test_size = 0.25, random_state = 21)


traindf_o = pd.concat([Xo_train,yo_train], axis = 1)
evaldf_o = pd.concat([Xo_eval,yo_eval], axis = 1)

traindf_r = pd.concat([Xr_train,yr_train], axis = 1)
evaldf_r = pd.concat([Xr_eval,yr_eval], axis = 1)

In [4]:
train_o = Dataset.from_pandas(traindf_o, split = 'train')
eval_o = Dataset.from_pandas(evaldf_o, split = 'eval')

train_r = Dataset.from_pandas(traindf_r, split = 'train')
eval_r = Dataset.from_pandas(evaldf_r, split = 'eval')

In [5]:
tokenizer = AutoTokenizer.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)
    
tokenized_train_o = train_o.map(tokenize_function, batched=True)
tokenized_eval_o = eval_o.map(tokenize_function, batched=True)

tokenized_train_r = train_r.map(tokenize_function, batched=True)
tokenized_eval_r = eval_o.map(tokenize_function, batched=True)

Map:   0%|          | 0/4218 [00:00<?, ? examples/s]

Map:   0%|          | 0/1406 [00:00<?, ? examples/s]

Map:   0%|          | 0/4235 [00:00<?, ? examples/s]

Map:   0%|          | 0/1406 [00:00<?, ? examples/s]

In [6]:
model_o = AutoModelForSequenceClassification.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis", num_labels=3)
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args_o = TrainingArguments(output_dir="checkpoints/test_trainer_o", evaluation_strategy="epoch", num_train_epochs=2)

trainer_o = Trainer(
    model=model_o,
    args=training_args_o,
    train_dataset=tokenized_train_o,
    eval_dataset=tokenized_eval_o,
    compute_metrics=compute_metrics,
)

trainer_o.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.792200,0.695829,0.700569
2,0.498800,0.794279,0.712660


TrainOutput(global_step=1056, training_loss=0.632978663300023, metrics={'train_runtime': 486.7197, 'train_samples_per_second': 17.332, 'train_steps_per_second': 2.17, 'total_flos': 554906197988352.0, 'train_loss': 0.632978663300023, 'epoch': 2.0})

In [7]:
trainer_o.save_model('models/obama_final')

In [8]:
model_r = AutoModelForSequenceClassification.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis", num_labels=3)

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args_r = TrainingArguments(output_dir="checkpoints/test_trainer_r", evaluation_strategy="epoch", num_train_epochs=3)

trainer_r = Trainer(
    model=model_r,
    args=training_args_r,
    train_dataset=tokenized_train_r,
    eval_dataset=tokenized_eval_r,
    compute_metrics=compute_metrics,
)

trainer_r.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.832000,0.934538,0.611664
2,0.571400,1.021661,0.585349
3,0.372800,1.470590,0.580370


TrainOutput(global_step=1590, training_loss=0.5800815174414676, metrics={'train_runtime': 749.6024, 'train_samples_per_second': 16.949, 'train_steps_per_second': 2.121, 'total_flos': 835713993058560.0, 'train_loss': 0.5800815174414676, 'epoch': 3.0})

In [9]:
trainer_r.save_model('models/romney_final')

# Load from checkpoints

In [ ]:
model_o = AutoModelForSequenceClassification.from_pretrained("checkpoints/test_trainer_o", num_labels=3)
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args_o = TrainingArguments(output_dir="test_trainer_o", evaluation_strategy="epoch", num_train_epochs=2)

trainer_o = Trainer(
    model=model_o,
    args=training_args_o,
    train_dataset=tokenized_train_o,
    eval_dataset=tokenized_test_o,
    compute_metrics=compute_metrics,
)

trainer_o.train()

In [ ]:
model_r = AutoModelForSequenceClassification.from_pretrained("checkpoints/test_trainer_r", num_labels=3)

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args_r = TrainingArguments(output_dir="checkpoints/test_trainer_r", evaluation_strategy="epoch", num_train_epochs=3)

trainer_r = Trainer(
    model=model_r,
    args=training_args_r,
    train_dataset=tokenized_train_r,
    eval_dataset=tokenized_eval_r,
    compute_metrics=compute_metrics,
)

trainer_r.train()